# Income Estimate By Naive Bayes

## Introduction
If we are given some collection data, how can we find a distribution (or more specifically, find the parameters of a particular class of distribution), that fit this data "well". One of the common solutions is to use maximum likelihood estimation. The basic idea of maximum likelihood estimation, is that we want to pick parameters θ that maximize the probaiblity of the observed data; in other words, we want to choose θ to solve the optimization problem. It can serve as a nice principle for how we fit parameters of distributions to data.

One simple algorithm that results from the integration of maximum likelihood estimation techniques is the Naive Bayes algorithm for classification. Naive Bayes is a class of simple classifier based on Bayes' Rule and strong (or naive) independence assumptions between features. In this project, we are going to implement a Naive Bayes Classifier for the Census Income Data Set from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/). The goal of the project is to predict whether a person makes over 50K a year based on 14 features.

## Installation
Before getting started, you'll need to install various packages that will be used in this project. It is strongly recommanded to use Anaconda since all the open source packages can be individually installed from the Anaconda repository. By calling

$ pip install [packages you want to install]

Or

$ conda install [packages you want to install]

Anaconda compiles and builds all the packages in the Anaconda repository itself. The packages required are listed as follow.

You can also refer to requirements.txt to figure out the required packages.



## Dataset Description

The dataset consists 32561 instances, each representing an individual. The goal is to predict whether a person makes over 50K a year based on 14 features. The features are:

| column | type | description |
| --- |:---:|:--- |
| age | continuous | trips around the sun to date
| final_weight | continuous | census weight attribute; constructed from the original census data |
| education_num | continuous | numeric education scale -- their maximum educational level as a number |
| capital_gain | continuous | income from investment sources |
| capital_loss | continuous | losses from investment sources |
| hours_per_week | continuous | number of hours worked every week |
| work_class | categorical | `Private`, `Self-emp-not-inc`, `Self-emp-inc`, `Federal-gov`, `Local-gov`, `State-gov`, `Without-pay`, `Never-worked` |
| education | categorical | `Bachelors`, `Some-college`, `11th`, `HS-grad`, `Prof-school`, `Assoc-acdm`, `Assoc-voc`, `9th`, `7th-8th`, `12th`, `Masters`, `1st-4th`, `10th`, `Doctorate`, `5th-6th`, `Preschool` |
| marital_status | categorical | `Married-civ-spouse`, `Divorced`, `Never-married`, `Separated`, `Widowed`, `Married-spouse-absent`, `Married-AF-spouse` |
| occupation | categorical | `Tech-support`, `Craft-repair`, `Other-service`, `Sales`, `Exec-managerial`, `Prof-specialty`, `Handlers-cleaners`, `Machine-op-inspct`, `Adm-clerical`, `Farming-fishing`, `Transport-moving`, `Priv-house-serv`, `Protective-serv`, `Armed-Forces` |
| relationship | categorical | `Wife`, `Own-child`, `Husband`, `Not-in-family`, `Other-relative`, `Unmarried.` |
| race | categorical | `White`, `Asian-Pac-Islander`, `Amer-Indian-Eskimo`, `Other`, `Black` |
| sex | categorical | `Female`, `Male` |
| native_country | categorical | (41 values not shown here) |

In [14]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import gzip


## Data Preparation

As the data is from UCI repository, it is already quite clean. However, some instances contain missing `occupation`, `native_country` or `work_class` (represented as ? in the CSV file) and these have to be discarded from the training set. Also, it would be better rename the `income` column as `label` since we are implementing a classification algorithm here. Replace the value with 1 if `income` is `>50K` and 0 otherwise.

In [15]:
def read_csv(fn):
    with gzip.open(fn, "rt", newline='', encoding="UTF-8") as file:
        return pd.read_csv(file)


# loads and processes data
def load_data(file_name="census.csv.gz"):
    df = read_csv(file_name)
    df = df[df['occupation'].eq("?").eq(False)]
    df = df[df['native_country'].eq("?").eq(False)]
    df = df[df['work_class'].eq("?").eq(False)]
    
    
    df.loc[df['income'].eq(">50K"), 'income'] = 1
    df.loc[df['income'] != 1, 'income'] = 0
    df.rename(columns = {"income":"label"}, inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df


    

## Overview of Naive Bayes classifier

Let $X_1, X_2, \ldots, X_k$ be the $k$ features of a dataset, with class label given by the variable $y$. A probabilistic classifier assigns the most probable class to each instance $(x_1,\ldots,x_k)$, as expressed by
$$ \hat{y} = \arg\max_y P(y\ \mid\ x_1,\ldots,x_k) $$

Using Bayes' theorem, the above *posterior probability* can be rewritten as
$$ P(y\ \mid\ x_1,\ldots,x_k) = \frac{P(y) P(x_1,\ldots,x_n\ \mid\ y)}{P(x_1,\ldots,x_k)} $$


Naive Bayes classifiers assume that the feature values are conditionally independent given the class label, that is,
$ P(x_1,\ldots,x_n\ \mid\ y) = \prod_{i=1}^{k}P(x_i\ \mid\ y) $. This strong assumption helps simplify the expression for posterior probability to
$$ P(y\ \mid\ x_1,\ldots,x_k) = \frac{P(y) \prod_{i=1}^{k}P(x_i\ \mid\ y)}{P(x_1,\ldots,x_k)} $$

For a given input $(x_1,\ldots,x_k)$, $P(x_1,\ldots,x_k)$ is constant. 

Thus, the class of a new instance can be predicted as:

$$\hat{y} = \arg\max_y P(y) \prod_{i=1}^{k}P(x_i\ \mid\ y)$$


Observe that this is the product of $k+1$ probability values, which can result in very small numbers. When working with real-world data, this often leads to an arithmetic underflow. We will instead be adding the logarithm of the probabilities:

$$\hat{y} = \arg\max_y \underbrace{\log P(y)}_\text{log-prior} + \underbrace{\sum_{i=1}^{k} \log P(x_i\ \mid\ y)}_\text{log-likelihood}$$

The rest of the sections deal with how each of these probability distributions -- $P(y), P(x_1\ \mid\ y), \ldots, P(x_k\ \mid\ y)$ -- are estimated from data.


## Feature Predictor

Naive Bayes classifiers are popular because we can independently model each feature and mix-and-match model types based on the prior knowledge. For example, we might know (or assume) that $(X_i|y)$ has some distribution, so we can directly use the probability density or mass function of the distribution to model $(X_i|y)$.

We are going to use two classes of likelihood models:
- Gaussian models, for continuous real-valued features (parameterized by mean $\mu$ and variance $\sigma$)
- Categorical models, for features in discrete categories (parameterized by $\mathbf{p} = <p_0,p_1\ldots>$, one parameter per category)



## Gaussian Feature Predictor

The Gaussian distribution is characterized by two parameters - mean $\mu$ and standard deviation $\sigma$:
$$ f_Z(z) = \frac{1}{\sqrt{2\pi}\sigma} \exp{(-\frac{(z-\mu)^2}{2\sigma^2})} $$

Given $n$ samples $z_1, \ldots, z_n$ from the above distribution, the MLE for mean and standard deviation are:
$$ \hat{\mu} = \frac{1}{n} \sum_{j=1}^{n} z_j $$

$$ \hat{\sigma} = \sqrt{\frac{1}{n} \sum_{j=1}^{n} (z_j-\hat{\mu})^2} $$


In [16]:

# Feature predictor for a normally distributed real-valued, continuous feature.
class GaussianPredictor:

    # constructor
    def __init__(self, k):
        self.k = k
        self.mu = []
        self.sigma = []

        
    # update predictor statistics (mu, sigma) for Gaussian distribution
    def fit(self, x, y):
        x, y = list(x), list(y)
        x, y = pd.Series(x), pd.Series(y)
        df = pd.concat([x, y], axis=1)
        df.rename(columns={0:'x',1:'y'}, inplace=True)
        exp = df.groupby(['y']).mean()
        exp = np.array(exp['x'])
        std = df.groupby(['y']).std(ddof=0)
        std = np.array(std['x'])        
        self.mu = exp
        self.sigma = std
        
        return self

    # log likelihood of feature values x according to each class               
    def partial_log_likelihood(self, x):
        result = []
        for loc, scale in list(zip(self.mu, self.sigma)):
            row = stats.norm.logpdf(x, loc, scale)
            result.append(row)
        
        return np.array(result)


def gaussian_pred(k):
    return GaussianPredictor(k)

The method in our GaussianPredictor class can be descriped as follow:

- `fit()`: Learn parameters for the likelihood model using an appropriate Maximum Likelihood Estimator.
- `partial_log_likelihood()`: Use the previously learnt parameters to compute the probability density or mass of a given feature value, and return the natural logarithm of this value.

We are going to construct our Gaussian feature predictor at last on the census income dataset by calling the previous two functions.

## Categorical Feature Predictor

The categorical distribution with $l$ possible values $\{\text A, \text B, \text C, \ldots\}$ is characterized by the probability distribution $\mathbf p$ over these values. ($\mathbf{p} = (p_0,\dots,p_{l-1})$ where $\sum\mathbf p = 1$.)

If $C$ is categorically distributed, the probability of observing a particular value $z$ is:

$$ \Pr(C=z; \mathbf{p}) = \begin{cases}
    p_0 & \text{ if } z=0
\\  p_1 & \text{ if } z=1
\\  \vdots
\\  p_{l-1} & \text{ if } z=(l-1)
\end{cases}$$

Given $n$ samples $z_1, \ldots, z_n$ from $C$, the smoothed Maximum Likelihood Estimator for $\mathbf p$ is:
$$ \hat{p_t} = \frac{\sum_{j=1}^{n} [z_j=t] + \alpha}{n + l\alpha} $$

The term in the numerator $\sum_{j=1}^{n} [z_j=t]$ is the number of times the value $t$ occurred in the sample. To avoid the zero-count problem, smoothing is done over all possible values that may be generated by $C$. 

We are going to write a predictor that learns a different categorical distribution $C_i$ for each of $k$ possible classes.



### Notice

  1. We maintain a dictionary from each possible input token (i.e. each value) to an array of length $k$ that contains $(\Pr(C_0=z), \Pr(C_1=z), ..., \Pr(C_{k-1}=z))$.
  2. For the purpose of smoothing, we assume that all distributions can produce each value. That is, the set of possible values is the same for all $C_i$.

In [17]:
# Feature predictor for a categorical feature.
class CategoricalPredictor:

    # constructor
    def __init__(self, k):
        self.k = k
        self.p = None
        

    # initializes the predictor statistics (p) for Categorical distribution
    def fit(self, x, y, alpha=1.):
        x, y = list(x), list(y)
        x, y, count = pd.Series(x), pd.Series(y), pd.Series([0]*len(x))
        df = pd.concat([x, y, count], axis=1)
        df = df.rename(columns={0:'x',1:'y',2:'count'})
        n = list(df.groupby(['y']).size())
        l = [len(df['x'].unique())] * self.k
        
        result = {}
        category = df['x'].unique()
        for c in category:
            result[c] = [0] * self.k
            
        # Get the count
        gps = df.groupby(['y', 'x']).groups
                
        for key in gps:
            gps[key] = len(gps[key])
        
        # Assign number to count list
        for k in gps:
            if k[1] in result:
                result[k[1]][k[0]] = gps[k]
        
        for k in result:
            nominator = np.array(result[k]) + alpha
            denominator = np.array(n) + np.array(l)*alpha
            result[k] = nominator / denominator
        
        self.p = result

        return self
    
    # log likelihood of feature values x according to each class
    def partial_log_likelihood(self, x):
        result = []
        for v in list(x):
            result.append(np.log(np.array(self.p[v])))
        
        return np.array(result).T
        

def categorical_pred(k):
    return CategoricalPredictor(k)


Same as Gaussian feature predictor, we are going to construct our categorical feature predictor at last on the census income dataset by calling `fit()` and `partial_log_likelihood()` functions.

## Income prediction

The final step is to construct a Naive Bayers classifier using the predictor class we wrote before. The methods to be implemented in the Naive Bayers class are descriped as follow:

- `__init__()`: Compute the log prior for each class and initialize the feature predictors (based on feature type). The smoothed prior for class $t$ is given by
$$ \text{prior}(t) = \frac{n_t + \alpha}{n + k\alpha} $$
where $n_t = \sum_{j=1}^{n} [y_j=t]$, (i.e., the number of times the label $t$ occurred in the sample), $n$ is the number fo entries in the sample, and $k$ is the number of label values. 
- `log_likelihood()`: Compute the sum of the log prior and partial log likelihoods for all features. Use it to predict the final class label.
- `predict()`: Use the output of log_likelihood to predict a class label; break ties by predicting the class with lower id.

We then feed the classifer with the income dataset, to predict the class that each instance might belongs to. We use the classification error rate to evaluate the accuracy of our prediction compared to the true class label.

In [18]:

# Naive Bayes classifier for a mixture of continuous and categorical attributes.
class NaiveBayesClassifier:

    # initialize predictors for each feature and compute class prior
    def __init__(self, df, alpha=1.):
        gps = df.groupby(['label']).groups
        for group in gps:
            gps[group] = len(gps[group])
        n = len(df['label'])
        k = len(gps)
        
        self.log_prior = []
        for group in gps:
            nominator = gps[group] + alpha
            denominator = n + k*alpha
            self.log_prior.append(np.log(nominator / denominator))
        self.log_prior = np.asarray(self.log_prior)
        
        self.predictor = {}
        labels = list(df['label'])
        feature_cols = list(df.drop("label", axis="columns").columns)
        for col in feature_cols:
            feature = list(df[col])
            if df[col].dtype == "object":
                predictor = CategoricalPredictor(k)
            elif df[col].dtype == "int64":
                predictor = GaussianPredictor(k)
            else:
                raise Exception()
            predictor.fit(feature, labels)
            self.predictor[col] = predictor
        
        
    # log_likelihood for input instances from log_prior and partial_log_likelihood of feature predictors
    def log_likelihood(self, x):
        array = np.zeros((len(self.log_prior), len(x)))
        cols = list(x.columns)
        for col in cols:
            if col != "label":
                array += self.predictor[col].partial_log_likelihood(x[col])
        result = self.log_prior[:,None] + array
        return result
        
              
    # predicts label for input instances, breaks ties in favor of the class with lower id.
    def predict(self, x):
        columns = x.columns
        if "label" in list(columns):
            x = x.drop("label", axis="columns")
        # Select the maximum likelyhood
        return self.log_likelihood(x).argmax(axis=0)
        

def naive_bayes(*args, **kwargs):
    return NaiveBayesClassifier(*args, **kwargs)


df = load_data()
cl = naive_bayes(df)
lp = cl.predict(df.drop("label", axis="columns"))

np.mean(np.array(df['label']) == np.array(lp))

0.8274318679132684

## Conclusion

Finally we achieve accuracy of 82% in predicting whether a person makes over 50K a year based on 14 features. Notice that one of the most important assumptions that Naive Bayes makes is that all of the features are independent. However, we haven't tested the correlation for our features. The accuracy could be further improved if we drop those columns that have high correlation with each other, or we can add an interaction feature to account for that correlation. Another potential issue that might occurs is overfitting. It is hard to determine whether 14 features is enough or we need add more. If the computation power is allowed, we may want to start trying some combinations starting from a certain number of features.

## References
1. Pandas: https://pandas.pydata.org/
2. Numpy: https://numpy.org/
3. Scipy: https://www.scipy.org/